In [1]:
!pip install syft

In [2]:
import torch as th
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(2, 20)
    self.fc2 = nn.Linear(20, 1)
  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x
model = Net()

def train():
  opt = optim.SGD(params=model.parameters(), lr=0.1)
  for iter in range(20):
    #erase prev gradient
    opt.zero_grad()
    #make prediction
    pred = model(data)
    #calculate how much we missed
    loss = ((pred - target)**2).sum()
    
    #figure out which weights caused to miss
    loss.backward()
    
    #change those weights
    opt.step()
    print(loss.data)
train()
    

tensor(1.3870)
tensor(2.4042)
tensor(6.5014)
tensor(3.5868)
tensor(1.1798)
tensor(0.3896)
tensor(0.2417)
tensor(0.1291)
tensor(0.0662)
tensor(0.0360)
tensor(0.0205)
tensor(0.0127)
tensor(0.0086)
tensor(0.0063)
tensor(0.0048)
tensor(0.0038)
tensor(0.0030)
tensor(0.0024)
tensor(0.0020)
tensor(0.0016)


In [3]:
model(data)

tensor([[-0.0099],
        [ 0.0120],
        [ 1.0223],
        [ 0.9762]], grad_fn=<AddmmBackward>)

Encrypt the model and **dfata** **
**

In [4]:
import syft as sy
hook = sy.TorchHook(th)
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

W0813 11:18:33.433005 140226367100800 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0813 11:18:33.455220 140226367100800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0813 11:18:34.258701 140226367100800 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0813 11:18:34.260280 140226367100800 base.py:646] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0813 11:18:34.261543 140226367100800 base.py:646] Worker me already exists. Replacing old worker which could cause          

In [0]:
encrypted_model = model.fix_precision().share(alice,bob, crypto_provider=secure_worker)

In [6]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:55598977359 -> alice:51900054397]
 	-> [PointerTensor | me:16242434097 -> bob:93588736090]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:30771562954 -> alice:59682572048]
 	-> [PointerTensor | me:55709572472 -> bob:81176310378]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:34161533928 -> alice:99407446147]
 	-> [PointerTensor | me:23752302079 -> bob:8335114858]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>[AdditiveSharingTensor]
 	-> [PointerTensor | me:44744990144 -> alice:48908705385]
 	-> [PointerTensor | me:14452086264 -> bob:39710179517]
 	*crypto provider: secure_worker*]

In [0]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [8]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:76112128648 -> alice:33549311040]
	-> [PointerTensor | me:76934702609 -> bob:89559486380]
	*crypto provider: secure_worker*

In [0]:
encrypted_prediction = encrypted_model(encrypted_data)

In [11]:
encrypted_prediction

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:60133072153 -> alice:61368962447]
	-> [PointerTensor | me:77268606941 -> bob:83467882551]
	*crypto provider: secure_worker*